In [3]:
import json
import pickle
from matplotlib import pyplot as plt
import numpy as np
from pymongo import MongoClient

## use shapely to determine if point is within bounds...
from shapely.geometry import Polygon as pg
from shapely.geometry import Point

In [4]:
with open("new_haven.geojson","r") as f:
    neighborhood_data=json.load(f)
with open("nh_bounds.pkl","r") as f:
    nh_bounds = pickle.load(f)

In [5]:
## get data from mongodb
client = MongoClient()
db=client.nh4
lngs=[]
lats=[]
ids=[]

for issue in db.issues.find():
    lats.append(issue["lat"])
    lngs.append(issue["lng"])
    ids.append(issue["id"])

In [6]:
## for each issue, determine which neighborhood the issue belongs to, 
## and weather or not the issue is in New Haven
q=0
hoods=[""]*len(lngs)
innh=[False]*len(lngs)
for i in range(len(neighborhood_data["features"])):
    neighborhood = neighborhood_data["features"][i]
    hood_bounds = neighborhood["geometry"]["coordinates"][0][0]
    name = neighborhood["properties"]['name']
    poly = pg(*[hood_bounds])
    poly2 = pg(*[nh_bounds])
    for i in range(len(lats)):
        if poly.contains(Point((lngs[i],lats[i]))):
            hoods[i]=name
        if poly2.contains(Point((lngs[i],lats[i]))):
            innh[i]=True

In [27]:
for i in range(24761,len(ids)):
    db.issues.update_one({"id":ids[i]},{"$set":{"neighborhood":hoods[i]}})

In [28]:
i

37878